У нас имеются 2 файла с метаданными:<br>
1) AirBase_v8_stations.csv - это старые метаданные, но здесь есть информация о соответствии станций городам 
<br>
2) PanEuropean_metadata.csv - это новые метаданные, здесь нет информации о соответствии станций городам
<br>

Нам нужно будет их объединить, для того чтобы получить метаданные с максимальной полной информацией.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_stat = pd.read_csv('AirBase_v8_stations.csv', sep="\t")
df_stat.info()
df_stat.sample(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8626 entries, 0 to 8625
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   station_european_code         8626 non-null   object 
 1   station_local_code            8374 non-null   object 
 2   country_iso_code              8626 non-null   object 
 3   country_name                  8626 non-null   object 
 4   station_name                  8626 non-null   object 
 5   station_start_date            8626 non-null   object 
 6   station_end_date              2514 non-null   object 
 7   type_of_station               8623 non-null   object 
 8   station_ozone_classification  3647 non-null   object 
 9   station_type_of_area          8626 non-null   object 
 10  station_subcat_rural_back     2785 non-null   object 
 11  street_type                   2887 non-null   object 
 12  station_longitude_deg         8626 non-null   float64
 13  sta

,station_european_code,station_local_code,country_iso_code,country_name,station_name,station_start_date,station_end_date,type_of_station,station_ozone_classification,station_type_of_area,station_subcat_rural_back,street_type,station_longitude_deg,station_latitude_deg,station_altitude,station_city,lau_level1_code,lau_level2_code,lau_level2_name,EMEP_station
1084,DEBE018,DEBE018,DE,Germany,B Schöneberg-Belziger Straße,1986-11-01,NaN,Background,NaN,urban,unknown,NaN,13.348775,52.485814,40,BERLIN,NaN,11000000,"Berlin, Stadt",no


В названиях всех скачанных файлов timeseries указан цифровой код станции  - local_code или local_id, а в наших метаданных коды станции содержат помимо цифр также и буквы. Нужно проверить, действительно ли в метаданных нет цифровых кодов станций.

In [3]:
df_meta = pd.read_csv('PanEuropean_metadata.csv', sep="\t")
df_meta.info()
df_meta.sample(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66163 entries, 0 to 66162
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Countrycode               66163 non-null  object 
 1   Timezone                  66163 non-null  object 
 2   Namespace                 66163 non-null  object 
 3   AirQualityNetwork         66163 non-null  object 
 4   AirQualityStation         66163 non-null  object 
 5   AirQualityStationEoICode  66163 non-null  object 
 6   AirQualityStationNatCode  66133 non-null  object 
 7   SamplingPoint             66163 non-null  object 
 8   SamplingProces            66160 non-null  object 
 9   Sample                    62077 non-null  object 
 10  AirPollutantCode          66163 non-null  object 
 11  ObservationDateBegin      59674 non-null  object 
 12  ObservationDateEnd        16417 non-null  object 
 13  Projection                66163 non-null  object 
 14  Longit

,Countrycode,Timezone,Namespace,AirQualityNetwork,AirQualityStation,AirQualityStationEoICode,AirQualityStationNatCode,SamplingPoint,SamplingProces,Sample,AirPollutantCode,ObservationDateBegin,ObservationDateEnd,Projection,Longitude,Latitude,Altitude,MeasurementType,AirQualityStationType,AirQualityStationArea,EquivalenceDemonstrated,MeasurementEquipment,InletHeight,BuildingDistance,KerbDistance
63679,TR,http://dd.eionet.europa.eu/vocabulary/aq/timez...,TR.UHKIA.AQD,NET-TR001A,STA-TR200241,TR0036A,TR200241,SPO-TR200241_5014,SPP-TR200241_5014_CD_TECORA_2.3_AUTO,SAM-TR200241_5014,http://dd.eionet.europa.eu/vocabulary/aq/pollu...,2007-02-16T00:00:00,NaN,EPSG:4979,29.08719,37.78156,382.0,active,background,urban,ref,NaN,3.5,-999.0,-999.0


К сожалению, после подробного и внимательного анализа файлов с метаданными выявилось отсутствие колонок, соответствующих идентификаторам 'localid". При этом, при просмотре нескольких случайно выбранных файлов какой-либо четкой закономерности нумерации и присваивания localid EoI-кодам выявлено не было. Так, например, EoI-коду DERP020 соответствует localid 3541, EoI-коду DERP019 - 3546, а DEBB032 - 3626. Также, в ходе проверки кодов в колонке AirQualityStationNatCode по папке с файлами не удалось найти ни одного совпадения с соответствием страны(например код 8145, который здесь приведен для Греции, используется в названиях файлов, соответствующих Германии).

In [4]:
print(list(df_meta[df_meta['Countrycode']=='NO']['AirQualityStationNatCode'].unique()))

['812', '9', '5', '428', '325', '425', '427', '426', '404', '429', '765', '7', '4', '856', '20', '223', '43', '15', '345', '263', '244', '243', '11', '12', '303', '283', '284', '14', '444', '203', '364', '384', '22', '464', '625', '525', '585', '304', '705', '830', '806', '808', '665', '666', '504', '605', '814', '827', '843', '163', '847', '853', '854', '852', '818', '896', '894', '885', '857', '815', '924', '959', '981', '1088', '985', '1091', '1087', '1137', '1136', '1140', '1142', '1143', '1150', '1147', '1145', '1146', '1141', '1148', '1144', '1152', '1161', '1162', '1164', '1163', '1159', '1167', '1168', '1172', '1174', '1175', '1173', '1155', '1169', '1170']


Здесь, например, для Норвегии максимальные значения 'AirQualityStationNatCode' являются четырехзначными. А в папке с загруженными файлами localid, указанные в названниях файлов, находятся в диапазоне с 28633-76736. Проверка еще нескольких случайно выбранных стран показала, что значения в данный колонке ни для одной страны не совпадают с localid в названиях файлов. Исходя из этого делаем вывод, что в таблице отсутствует информация по кодам localid. <br><br>
Попытка найти таблицу соответствия localid и StationEoICode на сайте eea.europa.eu и на других сайтах оказалась безуспешной. Поэтому нам нужно будет извлечь localid из названий файлов и добавить их в новые метаданные, чтобы можно было создавать датасеты для выбранных населенных пунктов.